<a href="https://colab.research.google.com/github/auramolina/Analitica-en-recursos-humanos/blob/main/giberto1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <font color='0C2054'> **Librerías** </font>

In [90]:
## Conectar el drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3 as sql #### para bases de datos sql

import warnings


warnings.filterwarnings("ignore")

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import silhouette_score, calinski_harabasz_score
!pip install kneed
from kneed import KneeLocator

from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans, DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.decomposition import PCA, FactorAnalysis

!pip install factor_analyzer
from factor_analyzer import FactorAnalyzer

## <font color='0c2054'> **IMPORTACIÓN DE DATOS** </font>

In [ ]:
# Lectura de los datos
data_general = pd.read_csv("https://raw.githubusercontent.com/auramolina/Analitica-en-recursos-humanos/main/Datos/general_data.csv", delimiter=';')
manager_survey = pd.read_csv("https://raw.githubusercontent.com/auramolina/Analitica-en-recursos-humanos/main/Datos/manager_survey.csv", delimiter=';')
retirement_info = pd.read_csv("https://raw.githubusercontent.com/auramolina/Analitica-en-recursos-humanos/main/Datos/retirement_info.csv", delimiter=';')
employee_survey = pd.read_csv("https://raw.githubusercontent.com/auramolina/Analitica-en-recursos-humanos/main/Datos/employee_survey_data.csv", delimiter=';')

## <font color='0c2054'> **Base de datos 1 - data_general** </font>




### <font color='0C2054'> **Selección de características** </font>

In [ ]:
data_general.sort_values(by=['EmployeeID'],ascending=1)

In [ ]:
## Frecuencia de la base de datos
for column in data_general.columns:
    print(f"Frecuencia en la columna: {column}")
    print(data_general[column].value_counts())
    print("\n")

En la búsqueda de variables que no aportan información útil, hemos analizado la frecuencia de los datos y hemos identificado que las variables EmployeeCount, Over18 y StandardHours contienen valores que se repiten en todos los registros. Al examinar estos datos, observamos que:

*Over18: El valor en esta columna siempre indica que la persona es mayor de 18 años, lo cual es un requisito para trabajar.

*StandardHours: El valor en esta columna siempre es 8 horas, lo cual es la jornada laboral estándar.

*EmployeeCount: El valor en esta columna es siempre 1, lo que representa el conteo de empleados para cada registro.

Dado que estos valores son constantes y no aportan información adicional sobre la variabilidad de los datos, hemos concluido que estas variables pueden ser descartadas, ya que no tienen utilidad para el análisis.

In [ ]:
# Eliminación de variables no significativas
data_general.drop(['EmployeeCount','Over18','StandardHours'], axis=1, inplace=True)

# Tamaño del dataset
data_general.shape

In [ ]:
# Información general del dataset
data_general.info()

In [ ]:
# Cambiar el tipo de dato de la variable fecha a formato fecha
data_general['InfoDate'] = pd.to_datetime(data_general['InfoDate'], format='%d/%m/%Y')
data_general['InfoDate']

In [ ]:
#observar los nulos de la base de datos
data_general.isnull().sum()

Al observar los nulos nos damo scuneta que no son significativos en contraste con la cantidad de datos de la base, por lo cual no se realiza imputación de datos y más adelante se evaluaran de nuevo

## <font color='0c2054'> **Base de datos 2 - manger_survery** </font>

### <font color='0C2054'> **Selección de características** </font>

In [ ]:
manager_survey.columns

In [ ]:
#Lectura de datos
manager_survey.sort_values(by=['EmployeeID'], ascending=1)

In [ ]:
# Información general del dataset
manager_survey.info()

In [ ]:
#observar los nulos de la base de datos
manager_survey.isnull().sum()

La base de dtaos no tiene nulos.

In [ ]:
# Convertir la columna 'infoDate' a tipo datetime
manager_survey['SurveyDate'] = pd.to_datetime(manager_survey['SurveyDate'])

## <font color='0c2054'> **Base de datos 3 - retirement_info** </font>

### <font color='0c2054'> **SELECCIÓN DE CARACTERÍSTICAS** </font>

In [ ]:
print(retirement_info.columns)

In [ ]:
#Lectura de datos
retirement_info.sort_values(by=['EmployeeID'],ascending=1)

In [ ]:
# Información general del dataset
retirement_info.info()

In [ ]:
## Frecuencia de la base de datos
for column in retirement_info.columns:
    print(f"Frecuencia en la columna: {column}")
    print(retirement_info[column].value_counts())
    print("\n")

Debido a que la variable 'Attrition' toma un solo valor se va a elimininar

In [ ]:
# Eliminación de variable
retirement_info.drop(['Attrition'], axis=1, inplace=True)


In [ ]:
#observar los nulos de la base de datos
retirement_info.isnull().sum()

Los datos nulos en la variable 'resignationReason' corresponden a los empleados que fueron despedidos, por lo cual no tienen una razón de retiro, por lo mismo los datos no se imputan.

In [ ]:
## Se realiza el cambio de formato de las fechas
retirement_info['retirementDate'] = pd.to_datetime(retirement_info['retirementDate'])

In [ ]:
retirement_info

## <font color='0c2054'> **Base de datos 4 - employee-survey** </font>

### <font color='0c2054'> **SELECCIÓN DE CARACTERÍSTICAS** </font>

In [ ]:
print(employee_survey.columns)

In [ ]:
#Lectura de datos
employee_survey.sort_values(by=['EmployeeID'],ascending=1)

In [ ]:
# Información general del dataset
employee_survey.info()

In [ ]:
#observar los nulos de la base de datos
employee_survey.isnull().sum()

In [ ]:
## Cambio del formato de fecha
employee_survey['DateSurvey'] = pd.to_datetime(employee_survey['DateSurvey'], format='%d/%m/%Y')
employee_survey['DateSurvey']

## <font color='0c2054'> **Bases de datos 2015 y 2016** </font>

La unión de las bases de datos y posterior división según los años se realizara con SQL

In [ ]:
## crear base de datos para unir las bases
conn = sql.connect("/content/drive/MyDrive/Analítica III/AA/Analitica-en-recursos-humanos/Datos/bd_emp.csv") ## crea una base de datos nueva donde se van a almacenar las otras.
cur=conn.cursor()

In [ ]:
data_general.to_sql('general_data', conn, if_exists='replace', index=False)
employee_survey.to_sql('employee_survey', conn, if_exists='replace', index=False)
manager_survey.to_sql('manager_survey', conn, if_exists='replace', index=False)
retirement_info.to_sql('retirement_info', conn, if_exists='replace', index=False)

In [ ]:
#llamar bases de datos
data_general = pd.read_sql('SELECT * FROM general_data', con=conn)
employee_survey = pd.read_sql('SELECT * FROM employee_survey', con=conn)
manager_survey = pd.read_sql('SELECT * FROM manager_survey', con=conn)
retirement_info = pd.read_sql('SELECT * FROM retirement_info', con=conn)

In [ ]:
#Cerrar la conección
conn.commit()
conn.close()

Con la consulta SQL se uniran las tablas creadas a partir de las bases de datos, y se obtendran nuevas tablas correspondientes a 2015 y 2016

In [ ]:
# Volver a conectar a la base de datos
conn = sql.connect("/content/drive/MyDrive/Analítica III/AA/Analitica-en-recursos-humanos/Datos/bd_emp.csv")
cur = conn.cursor()

# Definir la consulta SQL para unir las tablas y crear las nuevas tablas
consulta_sql = """
CREATE TABLE IF NOT EXISTS data_final AS
SELECT gd.*,
  es.EnvironmentSatisfaction, es.JobSatisfaction, es.WorkLifeBalance,
  ms.JobInvolvement, ms.PerformanceRating,
  ri.Attrition, ri.retirementDate, ri.retirementType, ri.resignationReason
FROM general_data AS gd
LEFT JOIN employee_survey AS es
ON gd.EmployeeID = es.EmployeeID AND strftime('%Y', gd.InfoDate) = strftime('%Y', es.DateSurvey)
LEFT JOIN manager_survey AS ms
ON gd.EmployeeID = ms.EmployeeID AND strftime('%Y', gd.InfoDate) = strftime('%Y', ms.SurveyDate)
LEFT JOIN (
  SELECT *
  FROM retirement_info
  WHERE strftime('%Y', retirementDate) = '2016'
) AS ri
ON gd.EmployeeID = ri.EmployeeID AND strftime('%Y', gd.InfoDate) = '2015';

CREATE TABLE IF NOT EXISTS datos_2015 AS
SELECT *
FROM data_final
WHERE strftime('%Y', InfoDate) = '2015'
AND (strftime('%Y', retirementDate) = '2016' OR retirementDate IS NULL);

CREATE TABLE IF NOT EXISTS datos_2016 AS
SELECT * ,
       NULL AS Attrition,
       NULL AS retirementDate,
       NULL AS retirementType,
       NULL AS resignationReason
FROM data_final
WHERE strftime('%Y', InfoDate) = '2016';
"""
cur.executescript(consulta_sql)

# Confirmar los cambios y cerrar la conexión
conn.commit()
conn.close()


In [ ]:
# Conectar nuevamente para cargar los datos en DataFrames
conn = sql.connect("/content/drive/MyDrive/Analítica III/AA/Analitica-en-recursos-humanos/Datos/bd_emp.csv")

datos_2015 = pd.read_sql("SELECT * FROM datos_2015", conn)
datos_2016 = pd.read_sql("SELECT * FROM datos_2016", conn)

# Cerrar la conexión a la base de datos
conn.close()

In [ ]:
## Se exportan los DataFrame a archivos csv
datos_2015.to_csv('/content/drive/MyDrive/Analítica III/AA/Analitica-en-recursos-humanos/Datos/df_2015', index=False)
datos_2016.to_csv('/content/drive/MyDrive/Analítica III/AA/Analitica-en-recursos-humanos/Datos/df_2016', index=False)